# Stock Price Forecasting:  Apple

In this project we use two time series analysis  methods to forecast Apple Stock. Time refers to the analysis of change in the trend of the data over a period of time,  in this case we use two methods the LSTM recurrent neural network and Facebooks Prophet algorithm to forecast Apple Stock. 

We then compare which method produces better results, LSTM or Prophet?

We use the data from 2017-2020 for training anf the 2021 data for forecasting!

In [1]:

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [2]:
import numpy  as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

from fbprophet import Prophet

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [ ]:
# #import external pandas_datareader library with alias of web
# import pandas_datareader as web
# import datetime
 
# #Start and end date to download Apple stock!
# start = datetime.datetime(2016, 12, 31)
# end = datetime.datetime(2021, 2, 8)
 
# #DataReader method name is case sensitive
# df = web.DataReader("aapl", 'yahoo', start, end)
#  modelPro=Prophet(yearly_seasonality=False,
#               weekly_seasonality=False,
#               daily_seasonality=False).add_seasonality(name="daily",
#                                                              period=10,
#                                                              fourier_order=100)
# #Save the dataframe as csv    
# df.to_csv('AppleStock2017Feb082021.csv')

In [ ]:
#Read all Apple stock data
appleData=pd.read_csv('AppleStock2017Feb082021.csv')

In [ ]:
appleData.head()

In [ ]:
appleData.tail()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(appleData['Close'],'r', label='Close')
plt.plot(appleData['Open'],'g',label='Open')
plt.plot(appleData['Low'],'b',label='Low')
plt.plot(appleData['High'],'g',label='High')
plt.plot(appleData['Volume']/2000000, '*-y', label='Volume [x2M]')
plt.ylabel('Daily price')
plt.xlabel('Days starting from 2017, Jan. 01')
plt.legend()
plt.title('Apple Stock')
plt.show()

It looks like high volumes are roughly associated with Close price. Can the Volume be a good predictor?!!

In [ ]:
appleData.plot('Date',['Open','Close','Low','High'], figsize=(12,6))

You can see in the above figure for the Close and Open that  the trend is highly non-linear and it is very difficult to capture the trend using this information. This is where the power of LSTM can be utilized.  Can the Prophet be useful? We will see!!!

# LSTM 

Let us know apply the LSTM method.

In [ ]:
# Scale the Close value  from zerpo to 1
appleClose=appleData['Close'].values.reshape(-1,1)
scaler = MinMaxScaler(feature_range = (0, 1))

apple_Close_scaled = scaler.fit_transform(appleClose)

In [ ]:
appleTest=appleData[appleData['Date']>='2020-01-01']
trainData=apple_Close_scaled[0:len(appleData)-len(appleTest)]
testData=apple_Close_scaled[len(appleData)-len(appleTest):]

print(f"Shape of training data is {trainData.shape} ")
print(f"Shape of testing  data is {testData.shape} ")

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(np.arange(0,len(trainData)),trainData,'k',label='Train')
plt.plot(np.arange(len(trainData),len(trainData)+len(testData)),testData,'r',label='Forecast')
plt.title('Apple stock train vs test')
plt.ylabel('Normalized Close')
plt.xticks([0, 200, 400, 600, 800, 1000],(appleData[['Date']].iloc[[0, 200, 400, 600, 800, 1000],:].values))
plt.xlabel('Days')
plt.legend()

In [ ]:
# Convert the training and testing to supervised 
def to_supervised(data, window):
    features_set = [] 
    labels = []
    for i in range(window, len(data)):
        features_set.append(data[i-window:i, 0])
        labels.append(data[i, 0])
    return np.array(features_set), np.array(labels).reshape(-1,1)

def to_reshape(dataX):
    data_xx=np.reshape(dataX,(dataX.shape[0],dataX.shape[1],1))
    print('The shape of of the data is: {}'.format(data_xx.shape))
    return data_xx


# LSTM model 

def LSTLModel():
    model=Sequential()
    model.add(LSTM(units=80, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=30, return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    
    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    return model

A preliminary investigation of the window range showed that a value bweetwen 60 and 70 produced resulted with the minimum RMSE of the apple stock. 
lesset window produced higher error because it doesn't provide for the LSTM sufficient information to forecast. Higher values also prouce lare RMS error persumably because it reduces the size of the test and training size. We selected window=60 in this experiment!

In [ ]:
# Now let run the model 

x_train, y_train=to_supervised(trainData,60)
x_test,y_test=to_supervised(testData,60)

x_train=to_reshape(x_train)
x_test=to_reshape(x_test)

model=LSTLModel()
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=200,batch_size=32,verbose=0)

In [ ]:
#plot histry
fig = plt.figure(figsize=(12,6))
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train loss', 'test loss'], loc='upper right')
plt.show()

# LSTM predictions

In [ ]:
predictions = model.predict(x_test,batch_size=32)
predictions = scaler.inverse_transform(predictions)
actual=scaler.inverse_transform(y_test)

RMSE=np.sqrt(mean_squared_error(predictions,actual))

plt.figure(figsize=(10,6))
plt.plot(np.arange(0,len(trainData)),scaler.inverse_transform(trainData),'k',label='Train')
plt.plot(np.arange(len(trainData),len(trainData)+len(testData)),scaler.inverse_transform(testData),'r',label='Test')
plt.plot(np.arange(len(trainData)+60,len(trainData)+60+len(predictions)),predictions,'b',label='Forecast')
plt.title('Apple stock train vs test')
plt.ylabel('Normalized Close')
plt.xticks([0, 200, 400, 600, 800, 1000],(appleData[['Date']].iloc[[0, 200, 400, 600, 800, 1000],:].values))
plt.xlabel('Days')
plt.legend()

plt.figure(figsize=(12,6))
plt.plot(actual, color='blue', label='Actual Apple Stock Price')
plt.plot(predictions , color='red', label='Predicted Apple Stock Price')
plt.title(f'Apple Stock Price Prediction,  RMSE= {np.round(RMSE,3)}')
plt.xlabel('Date')
plt.xticks([0, 50, 100, 150, 200],(appleData[['Date']].iloc[[0, 50, 100, 150, 200],:].values))
plt.ylabel('Apple Stock Price')
plt.legend()
plt.savefig('LSTMAppleStock.png', facecolor='w', format='png')
plt.show()

# Facebook's Prophet for Apple Stock forecasting

In [ ]:
#Prophet uses ds as the date name, and y, the variable we want to forecst
appleData.columns=['ds', 'High', 'Low', 'Open', 'y', 'Volume', 'Adj Close']

In [ ]:
appleData.head()

In [ ]:
appleData.head()

In [ ]:
appleData.head()

In [ ]:
testDataP=appleData[appleData['ds']>='2020-01-01']
trainDataP=appleData[appleData['ds']<'2020-01-01']
print(f"Shape of training data for Prophet is {trainDataP.shape}")
print(f"Shape of testing  data  for Prophet is {testDataP.shape}")

In [ ]:
modelPro=Prophet(yearly_seasonality=False,
              weekly_seasonality=False,
              daily_seasonality=False).add_seasonality(name="daily",
                                                             period=10,
                                                             fourier_order=100)

In [ ]:
modelPro.fit(trainDataP)

In [ ]:
future=modelPro.make_future_dataframe(freq='D',periods=len(testDataP))

In [ ]:
pred=modelPro.predict(future)

In [ ]:
fig=modelPro.plot(pred)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = modelPro.plot(pred)
a = add_changepoints_to_plot(fig.gca(), modelPro,pred)


In [ ]:
#plot the last 72 hour actual and forecast
RMSE=np.sqrt(mean_squared_error(pred['yhat'][754:].values,testDataP['y'].values))
plt.figure(figsize=(16,8))
plt.plot(pred['yhat'][754:].values,'-.r',label='forecast')
plt.plot(testDataP['y'].values,'-.k',label='actual')
plt.title('Prophet forecasted Apple Stock, RMSE: {}'.format(round(RMSE,3)))
plt.legend()
plt.rcParams.update({'font.size': 16,'lines.linewidth':1.5})
plt.savefig('ProphetAppleStock.png',format='png')